In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from selenium import webdriver
import os
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [3]:
class Zarascrape:

    def __init__ (self, url): # Declaro las variables self de la clase
        self.url = url 
        self.col_names = ["url", "description"]
        self.product_list = [] 
 
    def get_info(self, url): # saca la info de cada vestido en particular
        html = requests.get(url).content
        data = BeautifulSoup(html, "lxml")
        prod_title=data.find("h1",{"class":"product-name"})
        title=prod_title.text.replace(" Detalle","")
        prod_descr = data.find("p",{"class":"description"})
        prod_various = data.find("p",{"class":"product-color"})
        prod_color=prod_various.find("span",{"class": "_colorName"})
        prod_ref=prod_various.find("span",{"data-qa-qualifier": "product-reference"})
        prod_price=data.find_all("script",{"type":"application/ld+json"})
        for i in prod_price:
            price=i.text.split(',')[-2].lstrip('"price": ').replace('"','')
        prod_size=data.find_all("div",{"class":"size-select"})
        for all_size in prod_size:
            size=all_size.find_all("label", {'class':'product-size _product-size'}) #filtramos por tallas disponibles
            available_size=[s.text for s in size]
        prod_images=data.find_all("a",{"class":"_seoImg main-image"})
        prod_image=[e.get("href") for e in prod_images][0].lstrip("//")
        try:
            features=[title,prod_descr.text,prod_color.text,prod_ref.text, price, available_size,prod_image]
        except:
            features=["","","","","","",""]
        return features

    def get_attrs(webelement, attr='href'): 
        try: 
            return webelement.get_attribute(attr)
        except: 
            return ("")
    
    def get_list(self): # List of urls para vestidos
        driver = webdriver.Chrome((ChromeDriverManager().install()))
        driver.get(self.url)
        source_data = driver.page_source
        elem = driver.find_element_by_tag_name("body")

        no_of_pagedowns = 10

        res = []

        while no_of_pagedowns:
            print(no_of_pagedowns)
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            no_of_pagedowns-=1
        div = driver.find_elements_by_class_name('product-info-item')
        for i in div:
            b=i.find_elements_by_css_selector('a')
            for c in b:
                res.append(c)
        return [get_attrs(e, 'href') for e in res]

    def loop_it(self): # Iterates over the url root. 
        iterable = self.get_list()
        for e in iterable:
            print (len(self.product_list))
            self.product_list.append(self.get_info(e))
            time.sleep(3)
        return self.product_list
    
url_vestidos = 'https://www.zara.com/uk/en/woman-dresses-l1066.html?v1=1180427' # Scrapea sólo vestidos

# Assigns the Zarascrape class for dresses to an object
vestidos_ls = Zarascrape(url_vestidos)

# Launches the class method
data_vestidos = vestidos_ls.loop_it()

print(len(data_vestidos))
data_vestidos[:5]


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/almu/.wdm/chromedriver/2.46/mac64/chromedriver
10
9
8
7
6
5
4
3
2
1


NameError: name 'get_attrs' is not defined

In [ ]:
# Saves the df into a .csv
zara_base = pd.DataFrame(data_vestidos)

In [ ]:
zara_base.head()

In [ ]:
zara_base.shape

In [ ]:
# Saves the df into a .csv
zara_dresses = zara_base.to_csv("zara.csv", index=False)